In [ ]:
!nvidia-smi

Mon Apr  5 17:32:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install -Uqq fastcore onnx onnxruntime sentencepiece seqeval rouge-score
    !pip install -Uqq --no-deps fastai ohmeow-blurr
    !pip install -Uqq transformers datasets wandb

In [ ]:
from fastai.text.all import *
from fastai.callback.wandb import *

## Data

In [ ]:
# !wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
# !wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

In [ ]:
from datasets import load_dataset

In [ ]:
train_ds = load_dataset('squad', split='train')
valid_ds = load_dataset('squad', split='validation')

Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/0fd9e01360d229a22adfe0ab7e2dd2adc6e2b3d6d3db03636a51235947d4c6e9)
Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/0fd9e01360d229a22adfe0ab7e2dd2adc6e2b3d6d3db03636a51235947d4c6e9)


In [ ]:
train_ds[0]

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'title': 'University_of_Notre_Dame'}

In [ ]:
# train_ds = train_ds.select(range(1000))
len(train_ds)

87599

In [ ]:
def make_answer(sample):
    sample['answer_text'] = sample['answers']['text'][0]
    sample['answer_start'] = sample['answers']['answer_start'][0]
    return sample
train_ds = train_ds.map(make_answer, remove_columns=['answers'], num_proc=4)

In [ ]:
valid_ds = valid_ds.map(make_answer, remove_columns=['answers'], num_proc=4)

In [ ]:
train_df = train_ds.to_pandas()
valid_df = valid_ds.to_pandas()
train_df.head()

,answer_start,answer_text,context,id,question,title
0,515,Saint Bernadette Soubirous,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct...",5733be284776f41900661182,To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?,University_of_Notre_Dame
1,188,a copper statue of Christ,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct...",5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,University_of_Notre_Dame
2,279,the Main Building,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct...",5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame is beside to which structure?,University_of_Notre_Dame
3,381,a Marian place of prayer and reflection,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct...",5733be284776f41900661181,What is the Grotto at Notre Dame?,University_of_Notre_Dame
4,92,a golden statue of the Virgin Mary,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct...",5733be284776f4190066117e,What sits on top of the Main Building at Notre Dame?,University_of_Notre_Dame


In [ ]:
train_df.tail(1)

,answer_start,answer_text,context,id,question,title
87598,0,Kathmandu Metropolitan City,"Kathmandu Metropolitan City (KMC), in order to promote international relations has established an International Relations Secretariat (IRC). KMC's first international relationship was established in 1975 with the city of Eugene, Oregon, United States. This activity has been further enhanced by establishing formal relationships with 8 other cities: Motsumoto City of Japan, Rochester of the USA, Yangon (formerly Rangoon) of Myanmar, Xi'an of the People's Republic of China, Minsk of Belarus, and Pyongyang of the Democratic Republic of Korea. KMC's constant endeavor is to enhance its interacti...",5735d259012e2f140011a0a1,What is KMC an initialism of?,Kathmandu


In [ ]:
pd.concat??

In [ ]:
squad_df = pd.concat([train_df, valid_df])
squad_df['is_valid'] = squad_df.index > len(train_df)
squad_df.head(2)

,answer_start,answer_text,context,id,question,title,is_valid
0,515,Saint Bernadette Soubirous,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct...",5733be284776f41900661182,To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?,University_of_Notre_Dame,False
1,188,a copper statue of Christ,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct...",5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,University_of_Notre_Dame,False


## Training Setup

In [ ]:
model_name = 'distilbert-base-uncased'

max_len = 512
bs = 8
val_bs = 16

## Tracking

In [ ]:
import wandb

WANDB_NAME = f'squad-{model_name}-simple'
GROUP = f'SQUAD-{model_name}-simple'
NOTES = f'Simple finetuning {model_name} with RAdam lr=2e-5'
CONFIG = {}
TAGS =[model_name,'squad','radam','simple']

In [ ]:
wandb.init(reinit=True, project="vat", entity="fastai_community",
           name=WANDB_NAME, group=GROUP, notes=NOTES, tags=TAGS, config=CONFIG);

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Training

In [ ]:
def _to_device(e, device):
    if hasattr(e, 'to'): return e.to(device)
    elif isinstance(e, dict):
        for _, v in e.items():
            if hasattr(v, 'to'): v.to(device)
        return {k:(v.to(device) if hasattr(v, 'to') else v) for k, v in e.items()}

In [ ]:
@patch
def one_batch(self:Learner, i, b):
        self.iter = i
        b_on_device = tuple(_to_device(e, self.dls.device) for e in b) if self.dls.device is not None else b
        self._split(b_on_device)
        self._with_events(self._do_one_batch, 'batch', CancelBatchException)

In [ ]:
from transformers import *

from blurr.data.all import *
from blurr.modeling.all import *

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
task = HF_TASKS_AUTO.QuestionAnswering
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR_MODEL_HELPER.get_hf_objects(model_name, task=task, tokenizer_kwargs={'max_len':max_len})

In [ ]:
# pre_process_squad??

`blurr` preproc function. More to do here to use functionality of FastTokenizers as in huggingface example https://github.com/huggingface/transformers/blob/master/examples/question-answering/run_qa.py

In [ ]:
squad_df = squad_df.apply(partial(pre_process_squad, hf_arch=hf_arch, hf_tokenizer=hf_tokenizer), axis=1)
squad_df.head(1)

,answer_start,answer_text,context,id,question,title,is_valid,tokenized_input,tokenized_input_len,tok_answer_start,tok_answer_end
0,515,Saint Bernadette Soubirous,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct...",5733be284776f41900661182,To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?,University_of_Notre_Dame,False,"[[CLS], to, whom, did, the, virgin, mary, allegedly, appear, in, 1858, in, lou, ##rdes, france, ?, [SEP], architectural, ##ly, ,, the, school, has, a, catholic, character, ., atop, the, main, building, ', s, gold, dome, is, a, golden, statue, of, the, virgin, mary, ., immediately, in, front, of, the, main, building, and, facing, it, ,, is, a, copper, statue, of, christ, with, arms, up, ##rai, ##sed, with, the, legend, "", ve, ##ni, ##te, ad, me, om, ##nes, "", ., next, to, the, main, building, is, the, basilica, of, the, sacred, heart, ., immediately, behind, the, basilica, is, the, gr, ##ot...",176,130,138


In [ ]:
squad_df.tokenized_input_len.max()

870

In [ ]:
filtered_squad_df = squad_df[squad_df.tokenized_input_len <= max_len]

In [ ]:
trunc_strat = 'only_second' if (hf_tokenizer.padding_side == 'right') else 'only_first'

before_batch_tfm = HF_QABeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model,
                                             max_length=max_len, 
                                             truncation=trunc_strat, 
                                             tok_kwargs={ 'return_special_tokens_mask': True })

blocks = (
    HF_TextBlock(before_batch_tfm=before_batch_tfm, input_return_type=HF_QuestionAnswerInput), 
    CategoryBlock(vocab=list(range(max_len))),
    CategoryBlock(vocab=list(range(max_len)))
)

def get_x(x):
    return (x.question, x.context) if (hf_tokenizer.padding_side == 'right') else (x.context, x.question)

dblock = DataBlock(blocks=blocks, 
                   get_x=get_x,
                   get_y=[ColReader('tok_answer_start'), ColReader('tok_answer_end')],
                   splitter=ColSplitter(),
                   n_inp=1)

In [ ]:
dls = dblock.dataloaders(filtered_squad_df, bs=bs, val_bs=val_bs, shuffle=True)

In [ ]:
b = dls.one_batch()
b

({'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          ...,
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'),
  'cls_index': tensor([[0],
          [0],
          [0],
          [0],
          [0],
          [0],
          [0],
          [0]], device='cuda:0'),
  'input_ids': tensor([[ 101, 2000, 3183,  ...,    0,    0,    0],
          [ 101, 2054, 2001,  ...,    0,    0,    0],
          [ 101, 2070, 7829,  ...,    0,    0,    0],
          ...,
          [ 101, 2054, 2785,  ...,    0,    0,    0],
          [ 101, 2073, 2515,  ..., 4233, 1012,  102],
          [ 101, 2043, 2106,  ...,    0,    0,    0]], device='cuda:0'),
  'p_mask': tensor([[1, 0, 0,  ..., 1, 1, 1],
          [1, 0, 0,  ..., 1, 1, 1],
          [1, 0, 0,  ..., 1, 1, 1],
          ...,
          [1, 0, 0,  ..., 1, 1, 1],
          [1, 0, 0,  ..., 0, 0, 1],
    

In [ ]:
model = HF_BaseModelWrapper(hf_model)
learn = Learner(dls, 
                model,
                opt_func=RAdam,
                loss_func=MultiTargetLoss(),
                cbs=[HF_QstAndAnsModelCallback],
                splitter=hf_splitter).to_fp16()

learn.create_opt() 
learn.unfreeze()
learn.blurr_summary()

HF_BaseModelWrapper (Input shape: 8 x 213)
Layer (type)         Output Shape         Param #    Trainable 
                     8 x 213 x 768       
Embedding                                 23440896   True      
Embedding                                 393216     True      
LayerNorm                                 1536       True      
Dropout                                                        
Dropout                                                        
Linear                                    590592     True      
Linear                                    590592     True      
Linear                                    590592     True      
Linear                                    590592     True      
LayerNorm                                 1536       True      
Dropout                                                        
____________________________________________________________________________
                     8 x 213 x 3072      
Linear                      

In [ ]:
learn.show_training_loop()

Start Fit
   - before_fit     : [TrainEvalCallback, MixedPrecision, Recorder, ProgressCallback]
  Start Epoch Loop
     - before_epoch   : [Recorder, ProgressCallback]
    Start Train
       - before_train   : [TrainEvalCallback, Recorder, ProgressCallback]
      Start Batch Loop
         - before_batch   : [HF_QstAndAnsModelCallback, MixedPrecision]
         - after_pred     : [HF_QstAndAnsModelCallback, MixedPrecision]
         - after_loss     : [HF_QstAndAnsModelCallback, MixedPrecision]
         - before_backward: [MixedPrecision]
         - before_step    : [MixedPrecision]
         - after_step     : [MixedPrecision]
         - after_cancel_batch: []
         - after_batch    : [TrainEvalCallback, Recorder, ProgressCallback]
      End Batch Loop
    End Train
     - after_cancel_train: [Recorder]
     - after_train    : [Recorder, ProgressCallback]
    Start Valid
       - before_validate: [TrainEvalCallback, Recorder, ProgressCallback]
      Start Batch Loop
         - **CBs sa

### simple finetuning

In [ ]:
learn.fit_one_cycle(4, 2e-4, cbs=WandbCallback(log_preds=False, log_model=False))

Could not gather input dimensions


epoch,train_loss,valid_loss,time
0,4.165180,3.866700,22:53
1,3.529577,3.448304,22:52
2,2.317960,2.890562,23:11
3,1.347107,3.020964,23:12


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:760: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "


In [ ]:
# learner.blurr_predict